#### LSTM For EEG Data
- First, import everything we're gonna be using

In [7]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
torch.manual_seed(1)
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [227]:
class EEGLSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim = 4):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim)
        self.linear = nn.Linear(self.hidden_dim, self.output_dim)
        self.hidden = self.init_hidden(self.hidden_dim)
        
        # hidden layer init
    def init_hidden(self, hidden_dim):
        """Initialize the hidden state in self.hidden
        Dimensions are num_layers * minibatch_size * hidden_dim
        """
        return (autograd.Variable(torch.zeros(1, 1, hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, hidden_dim)))
    
    def forward(self, input):
        # input is 1000 dimensional for now
        input = input.reshape((1000,22)) # seq_len * input_size
        input = autograd.Variable(torch.FloatTensor(input))
        lstm_out, self.hidden = self.lstm(input, self.hidden)
        scores = self.linear(lstm_out)
        last_score = scores.data[-1]
        return autograd.Variable(last_score, requires_grad = True)
        
        
    

In [228]:
model = EEGLSTM(22, 20)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


In [86]:
# load the data
from load_data import EEGDataLoader
data_loader = EEGDataLoader()
X_train, y_train, X_test, y_test = data_loader.load_all_data()

In [229]:
sample = X_train[0][0]
sample.shape
scores = model(sample)
print(scores.shape)
scores = scores.data.numpy()
print(scores[0])


torch.Size([1, 4])
[ 0.24478126 -0.06899489 -0.17752998 -0.1222595 ]


In [230]:
# train the model
print(X_train.shape[1])
print(y_train.shape[1])

def get_target(y):
    label = y % 769
    vec = np.zeros(4)
    vec[label] = 1
    vec = vec.reshape((4,1))
    return autograd.Variable(torch.LongTensor(vec))

for epoch in range(500):
    for i in range(X_train.shape[1]):
        # get the data
        X_sample, y_sample = X_train[0][i], y_train[0][i]
        # clear out grads and re-init hidden state
        model.zero_grad()
        model.hidden = model.init_hidden(20)
        scores = model(X_sample)
        # compute the loss, grads, and update the parameters
        y_sample = [int(y_sample % 4)]
        target = autograd.Variable(torch.LongTensor(y_sample))
        loss = loss_function(scores, target)
        if i % 10 == 0:
            print(loss.data[0])
        loss.backward(retain_graph = True)
        optimizer.step()
    
        
        
        
        
    

238
238
1.4916436672210693
1.3306047916412354
1.429179072380066
1.1565102338790894
1.1571972370147705
1.2932132482528687
1.2321629524230957
1.310098648071289
1.570722222328186
1.628056287765503
1.6725482940673828
1.7501914501190186
1.4042288064956665
0.97127366065979
1.7506000995635986
1.3726789951324463
1.561462163925171
1.440813422203064
0.9572556018829346
1.792858600616455
1.5388904809951782
1.6464121341705322
1.2190728187561035
1.2579147815704346
1.4916436672210693
1.3306047916412354
1.429179072380066
1.1565102338790894
1.1571972370147705
1.2932132482528687
1.2321629524230957
1.310098648071289
1.570722222328186
1.628056287765503
1.6725482940673828
1.7501914501190186
1.4042288064956665
0.97127366065979
1.7506000995635986
1.3726789951324463
1.561462163925171
1.440813422203064
0.9572556018829346
1.792858600616455
1.5388904809951782
1.6464121341705322
1.2190728187561035
1.2579147815704346
1.4916436672210693
1.3306047916412354
1.429179072380066
1.1565102338790894
1.1571972370147705
1.29

KeyboardInterrupt: 